# Introduction

### Project initialization and setup

In [ ]:
# Importing all the libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as plt


In [ ]:
# Display options
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [ ]:
DB_PATH = "viewer_interactions.db"

try:
    conn = sqlite3.connect(DB_PATH)
    print("Connected successfully!")
except sqlite3.Error as e:
    print("Connection failed:", e)